In [0]:
%run ./config

In [0]:
from pyspark.sql.functions import max

In [0]:
loaded_dt = spark.read.format("csv")\
                .load("abfss://raw@kaniniproraw.dfs.core.windows.net/data/vehicle_instance")\
                .select("loaded_dt").agg(max("loaded_dt")).collect()[0][0]